In [25]:
import scaleapi
from scaleapi.tasks import TaskReviewStatus, TaskStatus

from requests import get
from io import BytesIO
from PIL import Image

# Phase 1 | Task Retrieval

In [26]:
API_KEY = "CHANGE_ME"
PROJECT_NAME = "Traffic Sign Detection"

client = scaleapi.ScaleClient(API_KEY)

tasks = client.get_tasks(
    project_name = PROJECT_NAME
)
input_tasks = []
# Iterating through the generator
for task in tasks:
    # Download task or do something!
    input_tasks.append(task.as_dict())

input_tasks[0]

{'task_id': '5f127f6f26831d0010e985e5',
 'created_at': '2020-07-18T04:49:51.129Z',
 'completed_at': '2020-07-18T05:03:26.917Z',
 'type': 'annotation',
 'status': 'completed',
 'instruction': 'Draw a tight box around each object.',
 'params': {'attachment': 'https://observesign.s3-us-west-2.amazonaws.com/traffic_sign_1.jpg',
  'attachment_type': 'image',
  'objects_to_annotate': ['traffic_control_sign',
   'construction_sign',
   'information_sign',
   'policy_sign',
   'non_visible_face'],
  'with_labels': True,
  'min_width': 0,
  'min_height': 0,
  'examples': [],
  'annotation_attributes': {'occlusion': {'description': 'What percent of the object is occluded?',
    'choices': ['0%', '25%', '50%', '75%', '100%']},
   'truncation': {'description': 'What percent of the object is truncated?',
    'choices': ['0%', '25%', '50%', '75%', '100%']},
   'background_color': {'description': 'What color is the background of the sign?',
    'choices': ['white',
     'red',
     'orange',
     'ye

# Phase 2 | Programmatic Quality Checks

## Response analysis

Here is the example of one annotation
```
{
    'label': 'policy_sign',
    'attributes': {'occlusion': '0%',
    'truncation': '0%',
    'background_color': 'white'},
    'uuid': '50fde071-d81d-4a21-8315-e831d5263b8f',
    'width': 12,
    'height': 17,
    'geometry': 'box',
    'left': 1173,
    'top': 460
}
```
We can check the quality of 'label', 'occlusion', 'truncation', 'background_color', 'width', 'height', 'left', 'top'.

- 'label', 'occlusion', 'truncation', 'background_color' must be values in the given options.

- 'width', 'height', 'left', 'top' represent the bounding box, which must be within the image and not too large or too small. 

In [27]:
def get_image_width_height(url: str):
    image_raw = get(url)
    image = Image.open(BytesIO(image_raw.content))
    return image.size

In [28]:
get_image_width_height(input_tasks[0]['params']['attachment'])

(1586, 891)

In [29]:
# develop a functio to check annotations' quality
def task_quality_check(input_task: dict):
    results = []
    width, height = get_image_width_height(input_task['params']['attachment'])
    
    for annotation in input_task['response']['annotations']:
        result = {}
        
        # add id
        result['uuid'] = annotation['uuid']
        
        # check label is in objects_to_annotate
        if annotation['label'] in input_task['params']['objects_to_annotate']:
            result['correct_label'] = True
        else:
            result['correct_label'] = False
            
        # check occlusion is in choices
        if annotation['attributes']['occlusion'] in input_task['params']['annotation_attributes']['occlusion']['choices']:
            result['correct_occlusion'] = True
        else:
            result['correct_occlusion'] = False
            
        # check truncation is in choices
        if annotation['attributes']['truncation'] in input_task['params']['annotation_attributes']['truncation']['choices']:
            result['correct_truncation'] = True
        else:
            result['correct_truncation'] = False            
           
        # check background_color is in choices
        if annotation['attributes']['background_color'] in input_task['params']['annotation_attributes']['background_color']['choices']:
            result['correct_background_color'] = True
        else:
            result['correct_background_color'] = False
            
        # check box is inside image        
        if annotation['left'] <= width and annotation['top'] <= height:
            result['box_inside_image'] = True
        else:
            result['box_inside_image'] = False
            
        # check box size. example only, which is best set between 2.5%~97.5% range of all boxes       
        if 0.0001 * width * height <= annotation['width'] * annotation['height'] <= 0.9999 * width * height:
            result['correct_box_size'] = True
        else:
            result['correct_box_size'] = False
            
        results.append(result)
    return results

In [30]:
task_quality_check(input_tasks[0])

[{'uuid': '50fde071-d81d-4a21-8315-e831d5263b8f',
  'correct_label': True,
  'correct_occlusion': True,
  'correct_truncation': True,
  'correct_background_color': True,
  'box_inside_image': True,
  'correct_box_size': True},
 {'uuid': 'c436c3c5-fb04-43b9-9679-30e0345c0837',
  'correct_label': True,
  'correct_occlusion': True,
  'correct_truncation': True,
  'correct_background_color': True,
  'box_inside_image': True,
  'correct_box_size': True},
 {'uuid': '0e8ce7a9-d203-4618-99be-8a63a10514e0',
  'correct_label': True,
  'correct_occlusion': True,
  'correct_truncation': True,
  'correct_background_color': True,
  'box_inside_image': True,
  'correct_box_size': True},
 {'uuid': 'ada7893a-3ee1-4d69-b06b-87240a9522cd',
  'correct_label': True,
  'correct_occlusion': True,
  'correct_truncation': True,
  'correct_background_color': True,
  'box_inside_image': True,
  'correct_box_size': True},
 {'uuid': '2deb7dd7-cf6d-4080-8a29-cdc085541b49',
  'correct_label': True,
  'correct_occlus

In [31]:
# develop a function to check all tasks' quality
def tasks_quality_check(input_tasks: list):
    results = []
    for input_task in input_tasks:
        result = {}
        
        # add id
        result['task_id'] = input_task['task_id']
        
        # add quality check
        result['task_quality'] = task_quality_check(input_task)
        
        results.append(result)
    return results

In [32]:
tasks_quality_check(input_tasks)

[{'task_id': '5f127f6f26831d0010e985e5',
  'task_quality': [{'uuid': '50fde071-d81d-4a21-8315-e831d5263b8f',
    'correct_label': True,
    'correct_occlusion': True,
    'correct_truncation': True,
    'correct_background_color': True,
    'box_inside_image': True,
    'correct_box_size': True},
   {'uuid': 'c436c3c5-fb04-43b9-9679-30e0345c0837',
    'correct_label': True,
    'correct_occlusion': True,
    'correct_truncation': True,
    'correct_background_color': True,
    'box_inside_image': True,
    'correct_box_size': True},
   {'uuid': '0e8ce7a9-d203-4618-99be-8a63a10514e0',
    'correct_label': True,
    'correct_occlusion': True,
    'correct_truncation': True,
    'correct_background_color': True,
    'box_inside_image': True,
    'correct_box_size': True},
   {'uuid': 'ada7893a-3ee1-4d69-b06b-87240a9522cd',
    'correct_label': True,
    'correct_occlusion': True,
    'correct_truncation': True,
    'correct_background_color': True,
    'box_inside_image': True,
    'corre

# Phase 3 | Reflection on Future Quality Checks

We can build a ML model to do quality check

1. Use the box 'width', 'height', 'left', 'top' to cut the image
2. Use the image cut as the input data and attributes as labels to train deep learning models to predict the quality of 'label', 'occlusion', 'truncation', 'background_color'.
    - Train a classification model to predict the probability of each label. 
    - Train a classification model to predict the probability of each occlusion option
    - Train a classification model to predict the probability of each truncation option
    - Train a classification model to predict the probability of each background_color option
3. For each attribute, if the given value's probability is smaller than a threshold, return an error message. If the given label's probability is greater than the threshold but it is not the highest probability, return an warning message.